In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !unzip /content/MIR-Phase3-2-20240628T020732Z-001.zip

In [3]:
import pandas as pd

ratings = pd.read_csv('/home/alireza/Repos/IMDB-IR-System/Logic/core/recommender_system/MIR-Phase3-2/ratings.csv', sep='\t', encoding='latin-1',
                      usecols=['user_id', 'movie_id', 'user_emb_id', 'movie_emb_id', 'rating'])

movies = pd.read_csv('/home/alireza/Repos/IMDB-IR-System/Logic/core/recommender_system/MIR-Phase3-2/movies.csv', sep='\t', encoding='latin-1',
                     usecols=['movie_id', 'title', 'genres'])

In [4]:
movies.head(5)

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings.head(5)

,user_id,movie_id,rating,user_emb_id,movie_emb_id
0,1,1193,5,0,1192
1,1,661,3,0,660
2,1,914,3,0,913
3,1,3408,4,0,3407
4,1,2355,5,0,2354


### Content Based

**Objective**: Build a Content-Based Recommendation system that computes similarity between movies based on movie genres. It will suggest movies that are most similar to a particular movie based on its genre.

**Dataset**:
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Preprocess Data**:
   - Split the `genres` column into separate genre strings.
   - Fill any missing values in the `genres` column and convert it to string.

2. **Compute TF-IDF Matrix**:
   - Use `TfidfVectorizer` to transform the genres into a TF-IDF matrix.

3. **Calculate Cosine Similarity**:
   - Compute cosine similarity between the TF-IDF matrices of movies.

4. **Recommend Movies**:
   - Create a function that gets movie recommendations based on the cosine similarity score of movie genres.

In [6]:
movies['genres'] = movies['genres'].str.split('|')
movies['genres'] = movies['genres'].fillna("").astype('str')

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2), min_df=1e-10, stop_words='english')
tfidf_matrix = tf.fit_transform(movies['genres'])

In [8]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [9]:
def genre_recommendations(title, n=10):
    index = movies[movies.title == title].index
    sim_scores = list(enumerate(cosine_sim[index].flatten()))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    chosen = [i[0] for i in sim_scores[1: n + 1]]
    return movies.iloc[chosen]['title']

In [10]:
# Example
genre_recommendations('GoldenEye (1995)')

345     Clear and Present Danger (1994)
543           Surviving the Game (1994)
724                    Rock, The (1996)
788                     Daylight (1996)
825               Chain Reaction (1996)
978                 Maximum Risk (1996)
1467                    Anaconda (1997)
1513                     Con Air (1997)
1693                   Firestorm (1998)
3686          Perfect Storm, The (2000)
Name: title, dtype: object


### Collaborative Filtering

**Objective**: Implement a collaborative filtering movie recommendation system to recommend movies to a user based on similar users' spreferences.

**Dataset**:

- **Ratings Data**: DataFrame with columns `user_id`, `movie_id`, and `rating`.
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Preprocess Data**:
   - Merge `ratings` and `movies` DataFrames on `movie_id`.
   - Split `genres` column into separate rows.

2. **Create User-Genre Matrix**:
   - Create a matrix where rows are users and columns are genres.
   - Each cell represents the ratio of movies watched by the user in that genre to the total movies watched by the user.

3. **Calculate User Similarity**:
   - Use cosine similarity to calculate the similarity between users.

4. **Find Top N Similar Users**:
   - Find the top N users with the most similar preferences to the given user.

5. **Recommend Movies**:
   - Recommend movies that similar users have watched but the target user has not.
   - Return titles and genres of the top 10 recommended movies.

In [11]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

ratings = pd.read_csv('/home/alireza/Repos/IMDB-IR-System/Logic/core/recommender_system/MIR-Phase3-2/ratings.csv', sep='\t', encoding='latin-1',
                      usecols=['user_id', 'movie_id', 'user_emb_id', 'movie_emb_id', 'rating'])

movies = pd.read_csv('/home/alireza/Repos/IMDB-IR-System/Logic/core/recommender_system/MIR-Phase3-2/movies.csv', sep='\t', encoding='latin-1',
                     usecols=['movie_id', 'title', 'genres'])

In [12]:
data = pd.merge(ratings, movies, on='movie_id')

In [13]:
data.head(5)

,user_id,movie_id,rating,user_emb_id,movie_emb_id,title,genres
0,1,1193,5,0,1192,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,0,660,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,0,913,My Fair Lady (1964),Musical|Romance
3,1,3408,4,0,3407,Erin Brockovich (2000),Drama
4,1,2355,5,0,2354,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [14]:
data = data.assign(genres=data['genres'].str.split('|')).explode('genres')

In [15]:
data.head(5)

,user_id,movie_id,rating,user_emb_id,movie_emb_id,title,genres
0,1,1193,5,0,1192,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,0,660,James and the Giant Peach (1996),Animation
1,1,661,3,0,660,James and the Giant Peach (1996),Children's
1,1,661,3,0,660,James and the Giant Peach (1996),Musical
2,1,914,3,0,913,My Fair Lady (1964),Musical


In [16]:
user_genre_matrix = data.pivot_table(index='user_id', columns='genres', aggfunc='count').fillna(0).apply(lambda x: x / x.sum(), axis=1)

In [17]:
cosine_sim = linear_kernel(user_genre_matrix.values, user_genre_matrix.values)

In [18]:
def get_top_n_similar_users(user_id, n=5):
    idx = user_genre_matrix.index.tolist()
    idx.remove(user_id)
    sorted_idx = sorted(idx, key=lambda x: cosine_sim[x - 1][user_id - 1], reverse=True)
    return sorted_idx[: n]

In [19]:
def recommend_movies(user_id, n_similar_users=5, n_recommendations=10):
    top_users = get_top_n_similar_users(user_id, n=n_similar_users)
    user_movies = set(data[data.user_id == user_id]['title'])
    similar_user_movies = data[data.user_id.isin(top_users)]
    new_movies = similar_user_movies[~similar_user_movies['title'].isin(user_movies)]
    movie_ratings = new_movies.groupby('title')['rating'].mean().reset_index()
    top_movies = movie_ratings.sort_values('rating', ascending=False).head(n_recommendations)
    return movies[movies['title'].isin(top_movies['title'])][['title', 'genres']]


In [20]:
# Example
user_id = 1
recommended_movies = recommend_movies(user_id, n_similar_users=5, n_recommendations=10)

In [21]:
recommended_movies

,title,genres
104,Nobody Loves Me (Keiner liebt mich) (1994),Comedy|Drama
259,"Little Princess, A (1995)",Children's|Drama
358,"Jungle Book, The (1994)",Adventure|Children's|Romance
791,Harriet the Spy (1996),Children's|Comedy
826,Matilda (1996),Children's|Comedy
905,"Little Princess, The (1939)",Children's|Drama
997,"Love Bug, The (1969)",Children's|Comedy
1006,"20,000 Leagues Under the Sea (1954)",Adventure|Children's|Fantasy|Sci-Fi
1116,Manon of the Spring (Manon des sources) (1986),Drama
2016,101 Dalmatians (1961),Animation|Children's


### SVD (Singular Value Decomposition)



**Objective**: Implement an SVD-based recommendation system to recommend movies to users by decomposing the user-item interaction matrix into latent factors.

**Dataset**:
- **Ratings Data**: DataFrame with columns `user_id`, `movie_id`, and `rating`.
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Preprocess Data**:
   - Merge `ratings` and `movies` DataFrames on `movie_id`.
   - Create a user-item matrix where rows represent users, columns represent movies, and the values are the ratings.

2. **Decompose Matrix using SVD**:
   - Apply Singular Value Decomposition (SVD) to decompose the user-item matrix into three matrices: $U$, $\Sigma$, and $V^T$.

3. **Reconstruct Matrix**:
   - Reconstruct the user-item matrix using the top $k$ singular values to reduce dimensionality.

4. **Predict Ratings**:
   - Use the reconstructed matrix to predict ratings for all user-item pairs.

5. **Recommend Movies**:
   - Recommend the top 20 movies with the highest predicted ratings for a given user that the user hasn't rated yet.

In [22]:
ratings = pd.read_csv('/home/alireza/Repos/IMDB-IR-System/Logic/core/recommender_system/MIR-Phase3-2/ratings.csv', sep='\t', encoding='latin-1',
                      usecols=['user_id', 'movie_id', 'user_emb_id', 'movie_emb_id', 'rating'])

movies = pd.read_csv('/home/alireza/Repos/IMDB-IR-System/Logic/core/recommender_system/MIR-Phase3-2/movies.csv', sep='\t', encoding='latin-1',
                     usecols=['movie_id', 'title', 'genres'])

In [23]:
n_users = ratings.user_id.unique().shape[0]
n_movies = ratings.movie_id.unique().shape[0]
print('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_movies))

Number of users = 6040 | Number of movies = 3706


Fill na elements.

In [24]:
Ratings = ratings.pivot(index = 'user_id', columns ='movie_id', values = 'rating').fillna(0)
Ratings.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
# calculate rating matrix

In [26]:
U, sigma, Vt = np.linalg.svd(Ratings)
sigma = np.diag(sigma)

In [27]:
k = 5
all_user_predicted_ratings = U[:, : k] @ sigma[: k, : k] @ Vt[: k, :]

In [28]:
preds = pd.DataFrame(all_user_predicted_ratings)
preds.head()

,0,1,2,3,4,5,6,7,8,9,...,3696,3697,3698,3699,3700,3701,3702,3703,3704,3705
0,1.685631,0.263657,0.191705,0.052080,0.147503,-0.382070,0.305648,0.043749,-0.066583,0.003126,...,0.017119,0.004704,0.018998,-0.016375,-0.033261,0.361982,0.028646,-0.015979,0.010325,0.016832
1,0.920278,0.322620,0.157758,0.054196,0.078879,1.089879,0.114110,0.016767,0.091101,0.711662,...,0.061313,0.000338,0.003461,0.053719,0.045406,0.500822,0.314432,0.060185,0.023865,0.323659
2,1.315070,0.311035,0.131752,-0.004057,0.075612,0.039469,0.125531,0.031794,-0.005471,0.326450,...,0.006380,0.001062,0.016108,0.000466,-0.017706,0.280116,0.014621,-0.009830,-0.003576,0.008373
3,0.374267,0.081980,-0.067999,-0.084151,-0.098862,0.246925,-0.168155,-0.003152,0.010323,0.246342,...,0.001615,-0.002508,0.003722,0.002466,0.008823,0.037400,0.000286,0.000266,-0.008217,-0.002446
4,1.289982,-0.127576,0.076331,0.100355,0.001728,1.250257,0.109804,-0.008535,-0.003762,0.165069,...,0.137289,0.003026,-0.003312,0.059284,0.025030,1.031307,0.676036,0.086296,0.082662,0.676366


In [29]:
def recommend_movies(predictions, userID, movies, original_ratings, num_recommendations):

    user_row_number = userID - 1 # User ID starts at 1, not 0
    user_predictions = pd.DataFrame({
        'movie_id': np.arange(len(predictions.iloc[user_row_number])) + 1,
        'Predictions': predictions.iloc[user_row_number]
    })

    user_data = original_ratings[original_ratings.user_id == userID]

    # merfe movies data with user_data
    user_full = user_data.merge(movies, how='left', on='movie_id')


    print(f'User {userID} has already rated {user_full.shape[0]} movies.')
    print(f'Recommending highest {num_recommendations} predicted ratings movies not already rated.')

    recommendations = (movies[~movies['movie_id'].isin(user_full['movie_id'])]
                        .merge(pd.DataFrame(user_predictions).reset_index(), how='left', left_on='movie_id', right_on='movie_id')
                        .rename(columns={user_row_number: 'Predictions'})
                        .sort_values('Predictions', ascending=False)
                        .iloc[:num_recommendations, :-1]
                      )

    recommendations = movies[movies.movie_id.isin(recommendations.movie_id)]
    return user_full.sort_values('rating', ascending=False), recommendations

In [30]:
already_rated, predictions = recommend_movies(preds, 4375, movies, ratings, 20)

User 4375 has already rated 325 movies.
Recommending highest 20 predicted ratings movies not already rated.


In [31]:
ratings.head(20)

,user_id,movie_id,rating,user_emb_id,movie_emb_id
0,1,1193,5,0,1192
1,1,661,3,0,660
2,1,914,3,0,913
3,1,3408,4,0,3407
4,1,2355,5,0,2354
5,1,1197,3,0,1196
6,1,1287,5,0,1286
7,1,2804,5,0,2803
8,1,594,4,0,593
9,1,919,4,0,918


In [32]:
# Top 20 movies that User 4375 has rated
already_rated.head(20)

,user_id,movie_id,rating,user_emb_id,movie_emb_id,title,genres
195,4375,3250,5,4374,3249,Alive (1993),Drama
285,4375,175,5,4374,174,Kids (1995),Drama
199,4375,3257,5,4374,3256,"Bodyguard, The (1992)",Action|Drama|Romance|Thriller
65,4375,3809,5,4374,3808,What About Bob? (1991),Comedy
210,4375,2688,5,4374,2687,"General's Daughter, The (1999)",Drama|Thriller
270,4375,150,5,4374,149,Apollo 13 (1995),Drama
271,4375,2710,5,4374,2709,"Blair Witch Project, The (1999)",Horror
125,4375,349,5,4374,348,Clear and Present Danger (1994),Action|Adventure|Thriller
60,4375,288,5,4374,287,Natural Born Killers (1994),Action|Thriller
141,4375,47,5,4374,46,Seven (Se7en) (1995),Crime|Thriller


#### Evaluation

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

train_data, test_data = train_test_split(ratings, test_size=0.2)

train_ratings = train_data.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)
R_train = train_ratings.values
user_ratings_mean_train = np.mean(R_train, axis=1)
Ratings_demeaned_train = R_train - user_ratings_mean_train.reshape(-1, 1)

U_train, sigma_train, Vt_train = np.linalg.svd(Ratings_demeaned_train, full_matrices=False)
sigma_train = np.diag(sigma_train)

all_user_predicted_ratings_train = np.dot(np.dot(U_train, sigma_train), Vt_train) + user_ratings_mean_train.reshape(-1, 1)
preds_train = pd.DataFrame(all_user_predicted_ratings_train, columns=train_ratings.columns)

def predict_rating(user_id, movie_id):
    try:
        user_index = train_ratings.index.get_loc(user_id)
        movie_index = train_ratings.columns.get_loc(movie_id)
        pred = preds_train.iloc[user_index, movie_index]
        return pred
    except KeyError:
        return np.nan

test_data['predicted_rating'] = test_data.apply(lambda x: predict_rating(x['user_id'], x['movie_id']), axis=1)

test_data.dropna(inplace=True)

rmse = sqrt(mean_squared_error(test_data['rating'], test_data['predicted_rating']))
print(f'Root Mean Squared Error: {rmse}')


Root Mean Squared Error: 3.7522497485011588


### Nueral Network Model (Recommender Model)



**Objective**: Implement a Recommender model to recommend movies to a user based on similar users' preferences.

**Dataset**:
- **Ratings Data**: DataFrame with columns `user_id`, `movie_id`, and `rating`.
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Define Dataset and DataLoader**:
   - Create a custom PyTorch `Dataset` for ratings.
   - Create a DataLoader for batching and shuffling data.

2. **Define the Neural Network**:
   - Create a neural network with embedding layers for users and movies.

3. **Train the Model**:
   - Train the model using Mean Squared Error loss and Adam optimizer.
   - Save model checkpoints.

4. **Evaluate the Model**:
   - Calculate RMSE on the entire dataset.

5. **Predict Ratings for Unrated Movies**:
   - Predict and recommend top 10 unrated movies for a given user.

In [34]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from math import sqrt

In [35]:
# Define the dataset
class RatingsDataset(Dataset):
    def __init__(self, ratings):
        self.user_ids = ratings['user_id'].values
        self.movie_ids = ratings['movie_id'].values
        self.ratings = torch.tensor(ratings['rating'].values, dtype=torch.float)
        self.user_map = dict()
        for i, x in enumerate(np.unique(self.user_ids)):
            self.user_map[x] = i
        self.movie_map = dict()
        for i, x in enumerate(np.unique(self.movie_ids)):
            self.movie_map[x] = i

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        return self.user_map[self.user_ids[idx]], self.movie_map[self.movie_ids[idx]], self.ratings[idx]

# Define the neural network
class RecommenderNet(nn.Module):
    def __init__(self, num_users, num_movies, embedding_size=50):
        super(RecommenderNet, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_size)
        self.movie_embedding = nn.Embedding(num_movies, embedding_size)
        self.fc1 = nn.Linear(embedding_size * 2, 128)
        self.fc2 = nn.Linear(128, 1)
        self.dropout = nn.Dropout(0.2)
        self.relu = nn.ReLU()


    def forward(self, user, movie):
        user_embed = self.user_embedding(user)
        movie_embed = self.movie_embedding(movie)
        x = torch.cat([user_embed, movie_embed], dim=1)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Create the dataset and dataloader
ratings = pd.read_csv('/home/alireza/Repos/IMDB-IR-System/Logic/core/recommender_system/MIR-Phase3-2/ratings.csv', sep='\t', encoding='latin-1',
                      usecols=['user_id', 'movie_id', 'user_emb_id', 'movie_emb_id', 'rating'])

num_users = ratings['user_id'].nunique()
num_movies = ratings['movie_id'].nunique()

dataset = RatingsDataset(ratings)
dataloader = DataLoader(dataset, batch_size=4096, shuffle=True)


# Initialize the model, loss function and optimizer
model = RecommenderNet(num_users, num_movies, embedding_size=50)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Training loop
device = 'cuda' if torch.cuda.is_available() else 'cpu'
epochs = 10
model = model.to(device)
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for user, movie, rating in dataloader:
        optimizer.zero_grad()
        user = user.to(device)
        movie = movie.to(device)
        rating = rating.to(device)
        output = model(user, movie).squeeze()
        loss = criterion(output, rating)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * user.size(0)

    epoch_loss = running_loss / len(dataloader.dataset)
    print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    predictions = []
    actuals = []
    for user, movie, rating in dataloader:
        user = user.to(device)
        movie = movie.to(device)
        output = model(user, movie).squeeze().cpu()
        predictions.extend(output.tolist())
        actuals.extend(rating.tolist())

model = model.cpu()
torch.save(model, 'model.pth')
rmse = sqrt(mean_squared_error(actuals, predictions))
print(f'Root Mean Squared Error: {rmse:.4f}')


Epoch 1/10, Loss: 2.5220
Epoch 2/10, Loss: 1.1501
Epoch 3/10, Loss: 1.0373
Epoch 4/10, Loss: 0.9805
Epoch 5/10, Loss: 0.9474
Epoch 6/10, Loss: 0.9259
Epoch 7/10, Loss: 0.9110
Epoch 8/10, Loss: 0.9012
Epoch 9/10, Loss: 0.8919
Epoch 10/10, Loss: 0.8848
Root Mean Squared Error: 0.8967


In [36]:
# Example

sample_user_id = 1
sample_user_data = ratings[ratings['user_id'] == sample_user_id]
sample_dataset = RatingsDataset(sample_user_data)
sample_dataloader = DataLoader(sample_dataset, batch_size=1, shuffle=False)

print(f'Sample predictions for user ID {sample_user_id}:')
with torch.no_grad():
    for user, movie, rating in sample_dataloader:
        output = model(user, movie).squeeze()
        print(f'Movie ID: {movie.item()}, Predicted Rating: {output.item()}, Actual Rating: {rating.item()}')

Sample predictions for user ID 1:
Movie ID: 22, Predicted Rating: 3.177719831466675, Actual Rating: 5.0
Movie ID: 10, Predicted Rating: 3.9813833236694336, Actual Rating: 3.0
Movie ID: 14, Predicted Rating: 2.6659109592437744, Actual Rating: 3.0
Movie ID: 52, Predicted Rating: 2.8597631454467773, Actual Rating: 4.0
Movie ID: 40, Predicted Rating: 4.271118640899658, Actual Rating: 5.0
Movie ID: 23, Predicted Rating: 3.4785218238830566, Actual Rating: 3.0
Movie ID: 27, Predicted Rating: 4.432799816131592, Actual Rating: 5.0
Movie ID: 47, Predicted Rating: 3.201336622238159, Actual Rating: 5.0
Movie ID: 7, Predicted Rating: 3.288320779800415, Actual Rating: 4.0
Movie ID: 15, Predicted Rating: 4.13198184967041, Actual Rating: 4.0
Movie ID: 8, Predicted Rating: 2.9687936305999756, Actual Rating: 5.0
Movie ID: 16, Predicted Rating: 4.256205081939697, Actual Rating: 4.0
Movie ID: 41, Predicted Rating: 3.12568998336792, Actual Rating: 4.0
Movie ID: 48, Predicted Rating: 3.95505690574646, Actua

In [37]:
def predict_unrated_movies(user_id, model, ratings, movies):
    user_map = dataset.user_map
    movie_map = dataset.movie_map
    user_rated_movies = ratings[ratings['user_id'] == user_id]['movie_id'].values
    unrated_movies = ratings[~ratings['movie_id'].isin(user_rated_movies)]
    model.eval()
    user_tensor = torch.tensor([user_map[user_id]], dtype=torch.long)
    
    predictions = []
    for movie_id in unrated_movies['movie_id']:
        movie_tensor = torch.tensor([movie_map[movie_id]], dtype=torch.long)
        with torch.no_grad():
            predicted_rating = model(user_tensor, movie_tensor).item()
        predictions.append((movie_id, predicted_rating))

    # Sort predictions by rating in descending order and return the top 10
    predictions.sort(key=lambda x: x[1], reverse=True)
    top_10_predictions = predictions[:10]
    return top_10_predictions


# Example
user_id = 1
predictions = predict_unrated_movies(user_id, model, ratings, movies)
print(predictions)

[(904, 4.826308727264404), (904, 4.826308727264404), (904, 4.826308727264404), (904, 4.826308727264404), (904, 4.826308727264404), (904, 4.826308727264404), (904, 4.826308727264404), (904, 4.826308727264404), (904, 4.826308727264404), (904, 4.826308727264404)]


### GMM (Gaussian Mixture Model)

**Objective**: Use a Gaussian Mixture Model to analyze and cluster the click data based on the number of clicks from different locations, aiming to identify distinct patterns of user behavior across 10 countries.

**Dataset**:
- **Click Data**: DataFrame with columns `link_id`, `location`, and `number_of_clicks`.

**Steps**:

1. **Preprocess Data**:
   - Ensure the dataset contains 10 distinct countries.
   - Create a matrix where rows represent different links and columns represent the number of clicks from each country.
   - Normalize the number of clicks to account for different scales.

2. **Fit GMM**:
   - Apply a Gaussian Mixture Model (GMM) to the click data matrix to identify clusters of links with similar click patterns across different countries.

3. **Evaluate Model**:
   - Analyze the resulting model by calculating the log lokelihood, BIC and AIC metrics.


In [38]:
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt

file_path = '/home/alireza/Repos/IMDB-IR-System/Logic/core/recommender_system/MIR-Phase3-2/gmm-dataset.csv'
df = pd.read_csv(file_path).set_index('URL')


data = df.values
data = data / data.sum(axis=1)[:,None]

gmm = GaussianMixture(n_components=5)
gmm.fit(data)

log_likelihood = gmm.score(data) * len(data)
bic = gmm.bic(data)
aic = gmm.aic(data)

print(f"Log-Likelihood: {log_likelihood}")
print(f"BIC: {bic}")
print(f"AIC: {aic}")

Log-Likelihood: 2067.4870746905062
BIC: -2619.8731581909306
AIC: -3476.9741493810125
